# --------------------------------------------------------------------------------------------------------------
# Playas de ESPAÑA 2017 : limpieza de datos para TABLEAU (CSV --> EXCEL)
# --------------------------------------------------------------------------------------------------------------

## INTRODUCCIÓN

El objetvo de este Notebook es leer un **dataset de entrada** (en **formato CSV**) y generar un **dataset de salida** (en **formato Excel**) ejecutando las limpiezas de datos necesarias para que el dataset de salida pueda ser procesado por la herramienta de visualización **TABLEAU**.

Para ello, llevaremos a cabo los siguientes **pasos** :

1. **Selecionar un dataset** de ENTRADA público.
2. **Limpiar/transformar** los datos del dataset de ENTRADA generando un dataset de SALIDA que sea apto para ser visualizado a través de TABLEAU.
3. **Volcar** el dataset "limpio" del paso anterior en un **fichero con formato Excel**.


## Dataset de partida

Vamos a utilizar un dataset de las PLAYAS de ESPAÑA correspondiente a los datos obtenidos en el año 2017.


La fuente de datos original es la siguiente :

http://opendata.esri.es/datasets/84ddbc8cf4104a579d579f6441fcaa8a_0

con :

   + 3.511 playas
   + 80 atributos



## Formato del fichero de ENTRADA

El datset de entrada es un fichero en formato **CSV** y utiliza el carácter **","** cómo separador de columnas.


## Formato del fichero de SALIDA

A partir del fichero **CSV** de entrada se generará un fichero de SALIDA en formato **EXCEL** (con los datos del dataset limpiados y transformados según consideremos oportuno para su posterior visualizacíón a través de **TABLEAU**).

# A) Preprocesado de datos

Con el objetivo de que el dataset pueda ser analizado fácilmente por Tableau, se realizará la siguiente limpieza (preprocesado) de los datos del dataset : 

   1. Cambiar **nombres de COMUNIDADES AUTÓNOMAS** :  cambiar los nombres de las comunidades autónomas de forma que Tableau los pueda reconocer como datos geográficos geolocalizables.   
   2. Cambiar ** nombres de PROVINCIAS :** cambiar los nombres de las provincias de forma que Tableau los pueda reconocer como datos geográficos geolocalizables.
   3. Crear un **nuevo campo numérico con la longitud de la playa** : el campo que indica la longitud de la playa es un campo de tipo texto. Creamos un nuevo campo equivalente que sea de tipo numérico para poder hacer cálculos matemáticos con esas longitudes.

## A.1) Importar librerías

In [1]:
import pandas as pd

## A.2) Leer la fuente de datos CSV en un DataFrame (DataFrame de PLAYAS)

In [2]:
df_playas = pd.read_csv("./data/Playas_2017_Dataset.csv", sep=',',  index_col=False )

In [3]:
# Verificamos que el df tiene las mismas dimensiones que el fichero CSV leído :
#    - Columnas : 80
#    - Filas    : 3511
df_playas.shape   

(3511, 80)

In [4]:
num_playas = df_playas.shape[0]     # Calculamos el número de playas
print (num_playas)

3511


In [5]:
df_playas.head(3)           # Consultamos el contenido de las primeras 3 filas

,X,Y,OBJECTID,Comunidad,Provincia,Isla,Código_IN,Núcleo_de_Población,Web_munici,Identifica,...,Dirección,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM
0,-14.0487,28.5718,3001,Canarias,Las Palmas,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,3007,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
1,-14.0089,28.6729,3002,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3008,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
2,-14.0086,28.6771,3003,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3009,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,"59,8 km.",Bolos / Grava / Arena,Montaña,No,,-14.0086,28.6771,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...


In [6]:
df_playas.info()     # Consultamos las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3511 entries, 0 to 3510
Data columns (total 80 columns):
X                      3511 non-null float64
Y                      3511 non-null float64
OBJECTID               3511 non-null int64
Comunidad              3511 non-null object
Provincia              3511 non-null object
Isla                   3511 non-null object
Código_IN              3511 non-null int64
Núcleo_de_Población    3511 non-null object
Web_munici             3511 non-null object
Identifica             3511 non-null int64
Nombre                 3511 non-null object
Nombre_alt             3511 non-null object
Nombre_a_1             3511 non-null object
Descripción            3511 non-null object
Longitud               3511 non-null object
Anchura                3511 non-null object
Variación              3511 non-null object
Grado_ocup             3511 non-null object
Grado_urba             3511 non-null object
Paseo_mar              3511 non-null object
Tipo_paseo    

## A.3) Cambiar a nombres geolocalizables por TABLEAU
TABLEAU reconoce los nombres de las provincias/comunidades españolas como datos geográficos, sin embargo, para que ese reconocimiento sea posible, los nombres deben ser los correctos. Analizando las provincias/comunidades del dataset, se observa que hay que efectuar los siguientes cambios :

**a) COMUNIDADES AUTÓNOMAS :**

   - Asturias, Principado de --> Asturias
   - Cataluña/Catalunya      --> Cataluña
   - Comunitat Valenciana    --> Comunidad Valenciana
   - Murcia, Región de       --> Murcia
   - País Vasco/Euskadi      --> País Vasco
   
**b) PROVINCIAS :**

   - Alicante/Alacant  --> Alicante
   - Valencia/València --> Valencia 

### A.3.1) Cambiar nombres de COMUNIDADES AUTÓNOMAS

Consultamos los nombres de las **Comunidades Autónomas** que tiene el dataset.

In [7]:
df_playas["Comunidad"].head(num_playas).drop_duplicates()

0                     Canarias
50                   Andalucía
52          Cataluña/Catalunya
58        Comunitat Valenciana
62                     Galicia
131    Asturias, Principado de
180                  Cantabria
182         País Vasco/Euskadi
212              Illes Balears
355          Murcia, Región de
448                      Ceuta
452                    Melilla
Name: Comunidad, dtype: object

Modificamos los nombres de las **Comunidades Autónomas** por los nombres que entiende **Tableau**. 


In [8]:
df_playas ["Comunidad"] = df_playas ["Comunidad"].replace("Asturias, Principado de", "Asturias")
df_playas ["Comunidad"] = df_playas ["Comunidad"].replace("Cataluña/Catalunya",      "Cataluña")
df_playas ["Comunidad"] = df_playas ["Comunidad"].replace("Comunitat Valenciana",    "Comunidad Valenciana")
df_playas ["Comunidad"] = df_playas ["Comunidad"].replace("Murcia, Región de",       "Murcia")
df_playas ["Comunidad"] = df_playas ["Comunidad"].replace("País Vasco/Euskadi",      "País Vasco")

In [9]:
#
# Otra forma equivalente de hacerlo :
#
#   df_playas["Comunidad"][df_playas["Comunidad"] == "Asturias, Principado de"] = "Asturias"
#   df_playas["Comunidad"][df_playas["Comunidad"] == "Cataluña/Catalunya"]      = "Cataluña"
#   df_playas["Comunidad"][df_playas["Comunidad"] == "Comunitat Valenciana"]    = "Comunidad Valenciana"
#   df_playas["Comunidad"][df_playas["Comunidad"] == "Murcia, Región de"]       = "Murcia"
#   df_playas["Comunidad"][df_playas["Comunidad"] == "País Vasco/Euskadi"]      = "País Vasco"

### A.3.2) Cambiar nombres de PROVINCIAS

Consultamos los nombres de las **Provincias** que tiene el dataset.

In [10]:
df_playas["Provincia"].head(num_playas).drop_duplicates()

0                  Las Palmas
50                    Almería
52                  Barcelona
58         Castellón/Castelló
62                 Pontevedra
112                    Girona
116                  A Coruña
120                      Lugo
131                  Asturias
180                 Cantabria
181                 Tarragona
182                   Bizkaia
183          Alicante/Alacant
212             Illes Balears
248                     Cádiz
266    Santa Cruz de Tenerife
315                   Granada
317                  Gipuzkoa
325                    Huelva
348                    Málaga
355                    Murcia
434         Valencia/València
448                     Ceuta
452                   Melilla
Name: Provincia, dtype: object

Modificamos los nombres de las **Provincias** por los nombres que entiende **Tableau**. 


In [11]:
df_playas ["Provincia"] = df_playas ["Provincia"].replace("Alicante/Alacant",  "Alicante")
df_playas ["Provincia"] = df_playas ["Provincia"].replace("Valencia/València", "Valencia")

In [12]:
#
# Otra forma equivalente de hacerlo :
#
#   df_playas["Provincia"][df_playas["Provincia"] == "Alicante/Alacant"]  = "Alicante"
#   df_playas["Provincia"][df_playas["Provincia"] == "Valencia/València"] = "Valencia"

## A.4) Crear un DataFrame de PROVINCIAS

Creamos un DataFrame de **PROVINCIAS** (con su Comunidad Autónoma asociada) con las provincias/comunidades que aparecen en nuestro  dataset de playas.

Este DataFrame nos servirá para :
1. **Verificar** que el nombre de las Comunidades Autónomas/Provincias se ha modificado correctamente
2. **Crear una hoja Excel con 2 pestañas** (para ver cómo trata TABLEAU una hoja Excel con más de una pestaña)

In [13]:
df_provincias = df_playas[["Provincia",
                           "Comunidad"]].drop_duplicates()
df_provincias.head(5)

,Provincia,Comunidad
0,Las Palmas,Canarias
50,Almería,Andalucía
52,Barcelona,Cataluña
58,Castellón/Castelló,Comunidad Valenciana
62,Pontevedra,Galicia


In [14]:
df_provincias.shape                         # Calculamos las dimensiones del DataFrame

(24, 2)

In [15]:
num_provincias = df_provincias.shape[0]     # Calculamos el número de provincias
print (num_provincias)

24


In [16]:
df_provincias.head(num_provincias)          # Consultamos el contenido de TODO el DataFrame 

,Provincia,Comunidad
0,Las Palmas,Canarias
50,Almería,Andalucía
52,Barcelona,Cataluña
58,Castellón/Castelló,Comunidad Valenciana
62,Pontevedra,Galicia
112,Girona,Cataluña
116,A Coruña,Galicia
120,Lugo,Galicia
131,Asturias,Asturias
180,Cantabria,Cantabria


## A.5) Retocar el DataFrame de PLAYAS

### A.5.1) Renombrar columnas

Renombramos ciertas columnas del DataFrame de **PLAYAS** para aclarar nombres.

In [17]:
df_playas = df_playas.rename(columns = {"Nombre"              : "nom_playa", 
                                        "OBJECTID"            : "cod_playa", 
                                        "Núcleo_de_Población" : "localidad", 
                                        "Longitud"            : "longitud_playa_txt", 
                                        "Tipo_de_ar"          : "arena", 
                                        "Nudismo"             : "nudista", 
                                        "Bandera_az"          : "bandera_azul"})
df_playas.head(3)

,X,Y,cod_playa,Comunidad,Provincia,Isla,Código_IN,localidad,Web_munici,Identifica,...,Dirección,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM
0,-14.0487,28.5718,3001,Canarias,Las Palmas,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,3007,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
1,-14.0089,28.6729,3002,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3008,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
2,-14.0086,28.6771,3003,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3009,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,"59,8 km.",Bolos / Grava / Arena,Montaña,No,,-14.0086,28.6771,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...


In [18]:
df_playas.shape  #  Verifcamos que seguimos con los 3511 registros

(3511, 80)

### A.5.2) Nueva columna "longitud_playa" con longitud numérica

Para poder operar aritméticamente con la longitud de las playas, transformamos el campo textual de "longitud_playa_txt" de la longitud de la playa en un campo numérico, para ello :
   1. Creamos la correspondiente columna numérica (longitud_playa).
   2. Eliminamos el texto que sigue al número que indica la longitud de la playa.
   3. Quitamos los puntos de los miles.
   4. Convertimos el tipo "carácter" a "entero".


In [19]:
df_playas.insert (5, "longitud_playa", int("0"))
df_playas.head(3)

,X,Y,cod_playa,Comunidad,Provincia,longitud_playa,Isla,Código_IN,localidad,Web_munici,...,Dirección,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM
0,-14.0487,28.5718,3001,Canarias,Las Palmas,0,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
1,-14.0089,28.6729,3002,Canarias,Las Palmas,0,Fuerteventura,35014,La Oliva,http://www.laoliva.es,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
2,-14.0086,28.6771,3003,Canarias,Las Palmas,0,Fuerteventura,35014,La Oliva,http://www.laoliva.es,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,"59,8 km.",Bolos / Grava / Arena,Montaña,No,,-14.0086,28.6771,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...


In [20]:
i=0
for texto in df_playas ["longitud_playa_txt"]:
    texto_new = texto
    texto_new = texto_new.split(" ")[0]         # Eliminamos todo lo que sigue al número (el texto "metros")
    texto_new = texto_new.replace (".", "")     # Eliminamos los puntos (=> elimino los puntos de los miles)
    texto_new = texto_new.replace (" ", "")     # Eliminamos los espacios
    df_playas ["longitud_playa"][i] = int (texto_new) # Cambiamos el tipo del campo de char a int
    i = i+1

In [21]:
i

3511

In [22]:
df_playas.head(3)

,X,Y,cod_playa,Comunidad,Provincia,longitud_playa,Isla,Código_IN,localidad,Web_munici,...,Dirección,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM
0,-14.0487,28.5718,3001,Canarias,Las Palmas,230,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
1,-14.0089,28.6729,3002,Canarias,Las Palmas,1210,Fuerteventura,35014,La Oliva,http://www.laoliva.es,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
2,-14.0086,28.6771,3003,Canarias,Las Palmas,190,Fuerteventura,35014,La Oliva,http://www.laoliva.es,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,"59,8 km.",Bolos / Grava / Arena,Montaña,No,,-14.0086,28.6771,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...


In [23]:
df_playas.info()     # Consultamos las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3511 entries, 0 to 3510
Data columns (total 81 columns):
X                     3511 non-null float64
Y                     3511 non-null float64
cod_playa             3511 non-null int64
Comunidad             3511 non-null object
Provincia             3511 non-null object
longitud_playa        3511 non-null int64
Isla                  3511 non-null object
Código_IN             3511 non-null int64
localidad             3511 non-null object
Web_munici            3511 non-null object
Identifica            3511 non-null int64
nom_playa             3511 non-null object
Nombre_alt            3511 non-null object
Nombre_a_1            3511 non-null object
Descripción           3511 non-null object
longitud_playa_txt    3511 non-null object
Anchura               3511 non-null object
Variación             3511 non-null object
Grado_ocup            3511 non-null object
Grado_urba            3511 non-null object
Paseo_mar             3511 non-null

# B) Creamos el fichero Excel con los datos finales

Grabamos el resultado en un fichero **Microsoft Excel** donde cada dataset (**PROVINCIAS** y **PLAYAS**) estará en un hoja diferente.

Concretamente, el **fichero Excel de SALIDA** contendrá las siguientes hojas (datasets):

   1. **HOJA_PLAYAS** : playas de ESPAÑA y sus características a fecha de 2017.
   2. **HOJA_PROVINCIAS :** provincias costeras de España que tengan playas (con la Comunidad Autónoma a la que corresponde).

In [24]:
writer = pd.ExcelWriter("./data/Playas_2017_TABLEAU___Dataset.xlsx")
df_playas.to_excel    (writer, 'HOJA_PLAYAS')
df_provincias.to_excel(writer, 'HOJA_PROVINCIAS')
writer.save()

# FIN